In [1]:
import pickle
from urllib.request import urlopen
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
np.random.RandomState(42)

import seaborn as sns

#import unittest
from nose.tools import *

import time
import datetime as dt
import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm, skew, kurtosis

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV,cross_val_score 
from sklearn import preprocessing
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_log_error,mean_squared_error, make_scorer

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [3]:
data1 = pd.read_parquet(Path("data") / "test.parquet")

In [4]:
df1 = pd.read_csv('2020.csv')

In [5]:
df2 = pd.read_csv('2021.csv')

In [6]:
dates20 = ["2020-01-01",'2020-04-10','2020-04-13','2020-05-01','2020-05-08','2020-05-21','2020-05-31','2020-06-01','2020-07-14','2020-08-15','2020-11-01','2020-11-11','2020-12-25','2020-12-26']

In [7]:
dates21 = ["2021-01-01",'2021-04-02','2021-04-05','2021-05-01','2021-05-08','2021-05-13','2021-05-23','2021-05-24','2021-07-14','2021-08-15','2021-11-01','2021-11-11','2021-12-25','2021-12-26']

In [8]:
dates_list = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in dates20]
df1["date"] = pd.to_datetime(df1["date"])

In [9]:
df1['holiday'] = df1['date'].isin(dates_list)
df1.holiday = df1.holiday.replace([True, False], [1, 0])
df1.loc[:, "weekday"] = df1["date"].dt.weekday
df1['weekend'] = df1['weekday'].isin([5,6])
df1.weekend = df1.weekend.replace([True, False], [1, 0])
df1 = df1.drop('weekday', axis=1)
df1

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,holiday,weekend
0,2020-01-01,0,0.8,-0.1,94,NaN,NaN,170.0,5.4,NaN,1032.5,NaN,NaN,1,0
1,2020-01-01,1,-0.7,-1.3,96,0.0,NaN,200.0,9.4,NaN,1032.2,NaN,NaN,1,0
2,2020-01-01,2,-0.3,-0.6,98,0.0,NaN,190.0,7.6,NaN,1032.2,NaN,NaN,1,0
3,2020-01-01,3,0.0,-0.3,98,NaN,NaN,220.0,5.4,NaN,1031.9,NaN,NaN,1,0
4,2020-01-01,4,0.3,0.0,98,0.0,NaN,210.0,7.6,NaN,1031.6,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2020-12-31,19,4.0,0.2,76,0.0,NaN,300.0,7.6,NaN,1006.3,NaN,NaN,0,0
8652,2020-12-31,20,3.2,-0.1,79,0.0,NaN,310.0,5.4,NaN,1006.6,NaN,NaN,0,0
8653,2020-12-31,21,3.0,-0.3,79,NaN,NaN,330.0,7.6,NaN,1007.0,NaN,NaN,0,0
8654,2020-12-31,22,2.5,-0.4,81,0.0,NaN,280.0,5.4,NaN,1007.5,NaN,NaN,0,0


In [10]:
dates_list = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in dates21]
df2["date"] = pd.to_datetime(df2["date"])

In [11]:
df2['holiday'] = df2['date'].isin(dates_list)
df2.holiday = df2.holiday.replace([True, False], [1, 0])
df2.loc[:, "weekday"] = df2["date"].dt.weekday
df2['weekend'] = df2['weekday'].isin([5,6])
df2.weekend = df2.weekend.replace([True, False], [1, 0])
df2 = df2.drop('weekday', axis=1)
df2

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,Coco,holiday,weekend
0,2021-01-01,0,1.6,-0.8,84,NaN,NaN,280,7.6,NaN,1008.0,NaN,NaN,1,0
1,2021-01-01,1,1.4,-1.0,84,0.0,NaN,280,5.4,NaN,1008.2,NaN,NaN,1,0
2,2021-01-01,2,0.9,-1.2,86,0.0,NaN,280,5.4,NaN,1008.6,NaN,NaN,1,0
3,2021-01-01,3,0.8,-1.1,87,NaN,NaN,280,5.4,NaN,1008.9,NaN,NaN,1,0
4,2021-01-01,4,0.6,-1.3,87,0.0,NaN,310,3.6,NaN,1009.0,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021-12-31,19,11.9,10.3,90,0.0,NaN,250,5.4,NaN,1026.5,NaN,NaN,0,0
8756,2021-12-31,20,12.0,10.6,91,0.0,NaN,280,11.2,NaN,1026.6,NaN,NaN,0,0
8757,2021-12-31,21,12.2,10.9,92,0.0,NaN,280,7.6,NaN,1026.6,NaN,NaN,0,0
8758,2021-12-31,22,12.0,10.6,91,0.0,NaN,270,3.6,NaN,1026.7,NaN,NaN,0,0


In [12]:
df1.loc[:, "season"] = df1["date"].dt.month
df1.season = df1.season.replace([1,2,3,4,5,6,7,8,9,10,11,12], [3,3,3,0,0,1,1,1,1,2,2,3])
df1

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,holiday,weekend,season
0,2020-01-01,0,0.8,-0.1,94,NaN,NaN,170.0,5.4,NaN,1032.5,NaN,NaN,1,0,3
1,2020-01-01,1,-0.7,-1.3,96,0.0,NaN,200.0,9.4,NaN,1032.2,NaN,NaN,1,0,3
2,2020-01-01,2,-0.3,-0.6,98,0.0,NaN,190.0,7.6,NaN,1032.2,NaN,NaN,1,0,3
3,2020-01-01,3,0.0,-0.3,98,NaN,NaN,220.0,5.4,NaN,1031.9,NaN,NaN,1,0,3
4,2020-01-01,4,0.3,0.0,98,0.0,NaN,210.0,7.6,NaN,1031.6,NaN,NaN,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2020-12-31,19,4.0,0.2,76,0.0,NaN,300.0,7.6,NaN,1006.3,NaN,NaN,0,0,3
8652,2020-12-31,20,3.2,-0.1,79,0.0,NaN,310.0,5.4,NaN,1006.6,NaN,NaN,0,0,3
8653,2020-12-31,21,3.0,-0.3,79,NaN,NaN,330.0,7.6,NaN,1007.0,NaN,NaN,0,0,3
8654,2020-12-31,22,2.5,-0.4,81,0.0,NaN,280.0,5.4,NaN,1007.5,NaN,NaN,0,0,3


In [13]:
df2.loc[:, "season"] = df2["date"].dt.month
df2.season = df2.season.replace([1,2,3,4,5,6,7,8,9,10,11,12], [3,3,3,0,0,1,1,1,1,2,2,3])
df2

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,Coco,holiday,weekend,season
0,2021-01-01,0,1.6,-0.8,84,NaN,NaN,280,7.6,NaN,1008.0,NaN,NaN,1,0,3
1,2021-01-01,1,1.4,-1.0,84,0.0,NaN,280,5.4,NaN,1008.2,NaN,NaN,1,0,3
2,2021-01-01,2,0.9,-1.2,86,0.0,NaN,280,5.4,NaN,1008.6,NaN,NaN,1,0,3
3,2021-01-01,3,0.8,-1.1,87,NaN,NaN,280,5.4,NaN,1008.9,NaN,NaN,1,0,3
4,2021-01-01,4,0.6,-1.3,87,0.0,NaN,310,3.6,NaN,1009.0,NaN,NaN,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021-12-31,19,11.9,10.3,90,0.0,NaN,250,5.4,NaN,1026.5,NaN,NaN,0,0,3
8756,2021-12-31,20,12.0,10.6,91,0.0,NaN,280,11.2,NaN,1026.6,NaN,NaN,0,0,3
8757,2021-12-31,21,12.2,10.9,92,0.0,NaN,280,7.6,NaN,1026.6,NaN,NaN,0,0,3
8758,2021-12-31,22,12.0,10.6,91,0.0,NaN,270,3.6,NaN,1026.7,NaN,NaN,0,0,3


In [14]:
df2.loc[:, "is_night"] = df2["hour"]
#df2.season = df2.season.replace([1,2,3,4,5,6,7,8,9,10,11,12], [3,3,3,0,0,1,1,1,1,2,2,3])
df2.drop('is_night',axis=1)

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,Coco,holiday,weekend,season
0,2021-01-01,0,1.6,-0.8,84,NaN,NaN,280,7.6,NaN,1008.0,NaN,NaN,1,0,3
1,2021-01-01,1,1.4,-1.0,84,0.0,NaN,280,5.4,NaN,1008.2,NaN,NaN,1,0,3
2,2021-01-01,2,0.9,-1.2,86,0.0,NaN,280,5.4,NaN,1008.6,NaN,NaN,1,0,3
3,2021-01-01,3,0.8,-1.1,87,NaN,NaN,280,5.4,NaN,1008.9,NaN,NaN,1,0,3
4,2021-01-01,4,0.6,-1.3,87,0.0,NaN,310,3.6,NaN,1009.0,NaN,NaN,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021-12-31,19,11.9,10.3,90,0.0,NaN,250,5.4,NaN,1026.5,NaN,NaN,0,0,3
8756,2021-12-31,20,12.0,10.6,91,0.0,NaN,280,11.2,NaN,1026.6,NaN,NaN,0,0,3
8757,2021-12-31,21,12.2,10.9,92,0.0,NaN,280,7.6,NaN,1026.6,NaN,NaN,0,0,3
8758,2021-12-31,22,12.0,10.6,91,0.0,NaN,270,3.6,NaN,1026.7,NaN,NaN,0,0,3


In [15]:
df1['is_night'] = 0
df1.loc[(df1['hour'] < 8) | (df1['hour'] > 20), 'is_night'] = 1
df1

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,holiday,weekend,season,is_night
0,2020-01-01,0,0.8,-0.1,94,NaN,NaN,170.0,5.4,NaN,1032.5,NaN,NaN,1,0,3,1
1,2020-01-01,1,-0.7,-1.3,96,0.0,NaN,200.0,9.4,NaN,1032.2,NaN,NaN,1,0,3,1
2,2020-01-01,2,-0.3,-0.6,98,0.0,NaN,190.0,7.6,NaN,1032.2,NaN,NaN,1,0,3,1
3,2020-01-01,3,0.0,-0.3,98,NaN,NaN,220.0,5.4,NaN,1031.9,NaN,NaN,1,0,3,1
4,2020-01-01,4,0.3,0.0,98,0.0,NaN,210.0,7.6,NaN,1031.6,NaN,NaN,1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2020-12-31,19,4.0,0.2,76,0.0,NaN,300.0,7.6,NaN,1006.3,NaN,NaN,0,0,3,0
8652,2020-12-31,20,3.2,-0.1,79,0.0,NaN,310.0,5.4,NaN,1006.6,NaN,NaN,0,0,3,0
8653,2020-12-31,21,3.0,-0.3,79,NaN,NaN,330.0,7.6,NaN,1007.0,NaN,NaN,0,0,3,1
8654,2020-12-31,22,2.5,-0.4,81,0.0,NaN,280.0,5.4,NaN,1007.5,NaN,NaN,0,0,3,1


In [16]:
df2['is_night'] = 0
df2.loc[(df2['hour'] < 8) | (df2['hour'] > 20), 'is_night'] = 1
df2

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,Coco,holiday,weekend,season,is_night
0,2021-01-01,0,1.6,-0.8,84,NaN,NaN,280,7.6,NaN,1008.0,NaN,NaN,1,0,3,1
1,2021-01-01,1,1.4,-1.0,84,0.0,NaN,280,5.4,NaN,1008.2,NaN,NaN,1,0,3,1
2,2021-01-01,2,0.9,-1.2,86,0.0,NaN,280,5.4,NaN,1008.6,NaN,NaN,1,0,3,1
3,2021-01-01,3,0.8,-1.1,87,NaN,NaN,280,5.4,NaN,1008.9,NaN,NaN,1,0,3,1
4,2021-01-01,4,0.6,-1.3,87,0.0,NaN,310,3.6,NaN,1009.0,NaN,NaN,1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021-12-31,19,11.9,10.3,90,0.0,NaN,250,5.4,NaN,1026.5,NaN,NaN,0,0,3,0
8756,2021-12-31,20,12.0,10.6,91,0.0,NaN,280,11.2,NaN,1026.6,NaN,NaN,0,0,3,0
8757,2021-12-31,21,12.2,10.9,92,0.0,NaN,280,7.6,NaN,1026.6,NaN,NaN,0,0,3,1
8758,2021-12-31,22,12.0,10.6,91,0.0,NaN,270,3.6,NaN,1026.7,NaN,NaN,0,0,3,1


In [17]:
weather = pd.concat([df1, df2], ignore_index=True)
weather = weather.drop(columns=['snow', 'wpgt','tsun','coco','Coco'])

In [18]:
weather

,date,hour,temp,dwpt,rhum,prcp,wdir,wspd,pres,holiday,weekend,season,is_night
0,2020-01-01,0,0.8,-0.1,94,NaN,170.0,5.4,1032.5,1,0,3,1
1,2020-01-01,1,-0.7,-1.3,96,0.0,200.0,9.4,1032.2,1,0,3,1
2,2020-01-01,2,-0.3,-0.6,98,0.0,190.0,7.6,1032.2,1,0,3,1
3,2020-01-01,3,0.0,-0.3,98,NaN,220.0,5.4,1031.9,1,0,3,1
4,2020-01-01,4,0.3,0.0,98,0.0,210.0,7.6,1031.6,1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17411,2021-12-31,19,11.9,10.3,90,0.0,250.0,5.4,1026.5,0,0,3,0
17412,2021-12-31,20,12.0,10.6,91,0.0,280.0,11.2,1026.6,0,0,3,0
17413,2021-12-31,21,12.2,10.9,92,0.0,280.0,7.6,1026.6,0,0,3,1
17414,2021-12-31,22,12.0,10.6,91,0.0,270.0,3.6,1026.7,0,0,3,1


In [19]:
weather["hour"] = pd.to_timedelta(weather['hour'], unit='h')
weather['date'] = weather['date'] + weather['hour']
weather = weather.drop(columns=['hour'])
weather = weather.fillna(method='bfill')
weather

,date,temp,dwpt,rhum,prcp,wdir,wspd,pres,holiday,weekend,season,is_night
0,2020-01-01 00:00:00,0.8,-0.1,94,0.0,170.0,5.4,1032.5,1,0,3,1
1,2020-01-01 01:00:00,-0.7,-1.3,96,0.0,200.0,9.4,1032.2,1,0,3,1
2,2020-01-01 02:00:00,-0.3,-0.6,98,0.0,190.0,7.6,1032.2,1,0,3,1
3,2020-01-01 03:00:00,0.0,-0.3,98,0.0,220.0,5.4,1031.9,1,0,3,1
4,2020-01-01 04:00:00,0.3,0.0,98,0.0,210.0,7.6,1031.6,1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17411,2021-12-31 19:00:00,11.9,10.3,90,0.0,250.0,5.4,1026.5,0,0,3,0
17412,2021-12-31 20:00:00,12.0,10.6,91,0.0,280.0,11.2,1026.6,0,0,3,0
17413,2021-12-31 21:00:00,12.2,10.9,92,0.0,280.0,7.6,1026.6,0,0,3,1
17414,2021-12-31 22:00:00,12.0,10.6,91,0.0,270.0,3.6,1026.7,0,0,3,1


In [20]:
train_merged = data.merge(weather, on='date')

In [21]:
test_merged = data1.merge(weather, on='date')

In [22]:
def add_hour(df):
    """
    The function is extracting the hour of a timestamp into a new column.
    Input: pandas DataFrame that contains 'timestamp' column
    """
    
    df['hour'] = df['date'].dt.hour
    
    assert_true('hour' in set(df.columns))
    return df

In [23]:
def add_month(df):
    """
    The function is extracting the month of a timestamp into a new column.
    Input: pandas DataFrame that contains 'timestamp' column
    """
    
    df['month'] = df['date'].dt.month
    
    assert_true('month' in set(df.columns))
    return df

In [24]:
add_month(train_merged)
add_hour(train_merged)

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,...,prcp,wdir,wspd,pres,holiday,weekend,season,is_night,month,hour
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0.0,290.0,3.6,1019.9,0,0,1,1,9,2
1,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,2.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0.0,290.0,3.6,1019.9,0,0,1,1,9,2
2,100036718-104036718,39 quai François Mauriac NO-SE,100036718,39 quai François Mauriac,5.0,2020-09-01 02:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0.0,290.0,3.6,1019.9,0,0,1,1,9,2
3,100036718-103036718,39 quai François Mauriac SE-NO,100036718,39 quai François Mauriac,1.0,2020-09-01 02:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0.0,290.0,3.6,1019.9,0,0,1,1,9,2
4,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,0.0,2020-09-01 02:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,...,0.0,290.0,3.6,1019.9,0,0,1,1,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450622,100060178-101060178,90 Rue De Sèvres SO-NE,100060178,90 Rue De Sèvres,18.0,2021-08-09 22:00:00,2020-07-22,Y2H20052705,48.846380,2.315290,...,0.0,220.0,5.4,1017.8,0,0,1,1,8,22
450623,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,33.0,2021-08-09 22:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0.0,220.0,5.4,1017.8,0,0,1,1,8,22
450624,100063175-353277235,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,14.0,2021-08-09 22:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0.0,220.0,5.4,1017.8,0,0,1,1,8,22
450625,300014702-353245972,254 rue de Vaugirard NE-SO,300014702,254 rue de Vaugirard,21.0,2021-08-09 22:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,...,0.0,220.0,5.4,1017.8,0,0,1,1,8,22


In [25]:
add_month(test_merged)
add_hour(test_merged)

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,...,prcp,wdir,wspd,pres,holiday,weekend,season,is_night,month,hour
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2021-08-10 05:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0.2,210.0,5.4,1017.9,0,0,1,1,8,5
1,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,16.0,2021-08-10 05:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0.2,210.0,5.4,1017.9,0,0,1,1,8,5
2,100036718-104036718,39 quai François Mauriac NO-SE,100036718,39 quai François Mauriac,17.0,2021-08-10 05:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0.2,210.0,5.4,1017.9,0,0,1,1,8,5
3,100036718-103036718,39 quai François Mauriac SE-NO,100036718,39 quai François Mauriac,53.0,2021-08-10 05:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0.2,210.0,5.4,1017.9,0,0,1,1,8,5
4,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,10.0,2021-08-10 05:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,...,0.2,210.0,5.4,1017.9,0,0,1,1,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41603,100060178-101060178,90 Rue De Sèvres SO-NE,100060178,90 Rue De Sèvres,43.0,2021-09-09 23:00:00,2020-07-22,Y2H20052705,48.846380,2.315290,...,0.0,210.0,11.2,1012.2,0,0,1,1,9,23
41604,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,88.0,2021-09-09 23:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0.0,210.0,11.2,1012.2,0,0,1,1,9,23
41605,100063175-353277235,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,21.0,2021-09-09 23:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0.0,210.0,11.2,1012.2,0,0,1,1,9,23
41606,300014702-353245972,254 rue de Vaugirard NE-SO,300014702,254 rue de Vaugirard,40.0,2021-09-09 23:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,...,0.0,210.0,11.2,1012.2,0,0,1,1,9,23


In [26]:
def add_encode(df, column, max_value):
    """
    The function is encoding time series cyclical features with sin and cos.
    Input: 
    ---------
    df - pandas DataFrame
    column - column name
    max_value - column max value
    Output: 
    -----------
    -same dataframe with _sin and _cos columns added
    """
    assert_true(column in set(df.columns))
    
    df[column + '_sin'] = np.sin(2 * np.pi * df[column] / max_value)
    df[column + '_cos'] = np.cos(2 * np.pi * df[column] / max_value)
    
    assert_true((column + '_sin') in set(df.columns))
    assert_true((column + '_cos') in set(df.columns))
    return df

In [27]:
add_encode(train_merged,'hour',23)
add_encode(train_merged,'month',12)

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,...,holiday,weekend,season,is_night,month,hour,hour_sin,hour_cos,month_sin,month_cos
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0,0,1,1,9,2,0.519584,0.854419,-1.000000,-1.836970e-16
1,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,2.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0,0,1,1,9,2,0.519584,0.854419,-1.000000,-1.836970e-16
2,100036718-104036718,39 quai François Mauriac NO-SE,100036718,39 quai François Mauriac,5.0,2020-09-01 02:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0,0,1,1,9,2,0.519584,0.854419,-1.000000,-1.836970e-16
3,100036718-103036718,39 quai François Mauriac SE-NO,100036718,39 quai François Mauriac,1.0,2020-09-01 02:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0,0,1,1,9,2,0.519584,0.854419,-1.000000,-1.836970e-16
4,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,0.0,2020-09-01 02:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,...,0,0,1,1,9,2,0.519584,0.854419,-1.000000,-1.836970e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450622,100060178-101060178,90 Rue De Sèvres SO-NE,100060178,90 Rue De Sèvres,18.0,2021-08-09 22:00:00,2020-07-22,Y2H20052705,48.846380,2.315290,...,0,0,1,1,8,22,-0.269797,0.962917,-0.866025,-5.000000e-01
450623,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,33.0,2021-08-09 22:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0,0,1,1,8,22,-0.269797,0.962917,-0.866025,-5.000000e-01
450624,100063175-353277235,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,14.0,2021-08-09 22:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0,0,1,1,8,22,-0.269797,0.962917,-0.866025,-5.000000e-01
450625,300014702-353245972,254 rue de Vaugirard NE-SO,300014702,254 rue de Vaugirard,21.0,2021-08-09 22:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,...,0,0,1,1,8,22,-0.269797,0.962917,-0.866025,-5.000000e-01


In [28]:
add_encode(test_merged,'hour',23)
add_encode(test_merged,'month',12)

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,...,holiday,weekend,season,is_night,month,hour,hour_sin,hour_cos,month_sin,month_cos
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2021-08-10 05:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0,0,1,1,8,5,9.790841e-01,0.203456,-0.866025,-5.000000e-01
1,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,16.0,2021-08-10 05:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,...,0,0,1,1,8,5,9.790841e-01,0.203456,-0.866025,-5.000000e-01
2,100036718-104036718,39 quai François Mauriac NO-SE,100036718,39 quai François Mauriac,17.0,2021-08-10 05:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0,0,1,1,8,5,9.790841e-01,0.203456,-0.866025,-5.000000e-01
3,100036718-103036718,39 quai François Mauriac SE-NO,100036718,39 quai François Mauriac,53.0,2021-08-10 05:00:00,2017-07-12,Y2H17021629,48.834360,2.377000,...,0,0,1,1,8,5,9.790841e-01,0.203456,-0.866025,-5.000000e-01
4,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,10.0,2021-08-10 05:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,...,0,0,1,1,8,5,9.790841e-01,0.203456,-0.866025,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41603,100060178-101060178,90 Rue De Sèvres SO-NE,100060178,90 Rue De Sèvres,43.0,2021-09-09 23:00:00,2020-07-22,Y2H20052705,48.846380,2.315290,...,0,0,1,1,9,23,-2.449294e-16,1.000000,-1.000000,-1.836970e-16
41604,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,88.0,2021-09-09 23:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0,0,1,1,9,23,-2.449294e-16,1.000000,-1.000000,-1.836970e-16
41605,100063175-353277235,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,21.0,2021-09-09 23:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,...,0,0,1,1,9,23,-2.449294e-16,1.000000,-1.000000,-1.836970e-16
41606,300014702-353245972,254 rue de Vaugirard NE-SO,300014702,254 rue de Vaugirard,40.0,2021-09-09 23:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,...,0,0,1,1,9,23,-2.449294e-16,1.000000,-1.000000,-1.836970e-16


In [29]:
data_tr = train_merged.sort_values(["date", "counter_name"])
y_train = data_tr["log_bike_count"].values
X_train = data_tr.drop(["log_bike_count", "bike_count"], axis=1)

In [30]:
data_ts = test_merged.sort_values(["date", "counter_name"])
y_test = data_ts["log_bike_count"].values
X_test = data_ts.drop(["log_bike_count", "bike_count"], axis=1)

In [31]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450627 entries, 1963 to 434002
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 450627 non-null  category      
 1   counter_name               450627 non-null  category      
 2   site_id                    450627 non-null  int64         
 3   site_name                  450627 non-null  category      
 4   date                       450627 non-null  datetime64[ns]
 5   counter_installation_date  450627 non-null  datetime64[ns]
 6   counter_technical_id       450627 non-null  category      
 7   latitude                   450627 non-null  float64       
 8   longitude                  450627 non-null  float64       
 9   temp                       450627 non-null  float64       
 10  dwpt                       450627 non-null  float64       
 11  rhum                       450627 non-null  int64

In [33]:
X_train['date'].unique()

array(['2020-09-01T01:00:00.000000000', '2020-09-01T02:00:00.000000000',
       '2020-09-01T03:00:00.000000000', ...,
       '2021-08-09T21:00:00.000000000', '2021-08-09T22:00:00.000000000',
       '2021-08-09T23:00:00.000000000'], dtype='datetime64[ns]')

In [34]:
X_train["counter_installation_date"] = pd.to_datetime(X_train["counter_installation_date"], format="%Y-%m-%d")

In [35]:
X_train

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,counter_technical_id,latitude,longitude,temp,...,holiday,weekend,season,is_night,month,hour,hour_sin,hour_cos,month_sin,month_cos
1963,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,13.6,...,0,0,1,1,9,1,2.697968e-01,0.962917,-1.000000,-1.836970e-16
1964,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,13.6,...,0,0,1,1,9,1,2.697968e-01,0.962917,-1.000000,-1.836970e-16
1948,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,13.6,...,0,0,1,1,9,1,2.697968e-01,0.962917,-1.000000,-1.836970e-16
1949,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,13.6,...,0,0,1,1,9,1,2.697968e-01,0.962917,-1.000000,-1.836970e-16
1996,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2020-09-01 01:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,13.6,...,0,0,1,1,9,1,2.697968e-01,0.962917,-1.000000,-1.836970e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434040,100057329-103057329,Totem 85 quai d'Austerlitz SE-NO,100057329,Totem 85 quai d'Austerlitz,2021-08-09 23:00:00,2020-02-18,YTH19111508,48.842010,2.367290,18.4,...,0,0,1,1,8,23,-2.449294e-16,1.000000,-0.866025,-5.000000e-01
434041,100057380-104057380,Totem Cours la Reine E-O,100057380,Totem Cours la Reine,2021-08-09 23:00:00,2020-02-11,YTH19111509,48.864620,2.314440,18.4,...,0,0,1,1,8,23,-2.449294e-16,1.000000,-0.866025,-5.000000e-01
434042,100057380-103057380,Totem Cours la Reine O-E,100057380,Totem Cours la Reine,2021-08-09 23:00:00,2020-02-11,YTH19111509,48.864620,2.314440,18.4,...,0,0,1,1,8,23,-2.449294e-16,1.000000,-0.866025,-5.000000e-01
434001,100042374-110042374,Voie Georges Pompidou NE-SO,100042374,Voie Georges Pompidou,2021-08-09 23:00:00,2017-12-15,Y2H21025335,48.848400,2.275860,18.4,...,0,0,1,1,8,23,-2.449294e-16,1.000000,-0.866025,-5.000000e-01


In [36]:
X_train['counter_installation_date'].unique()

array(['2018-12-07T00:00:00.000000000', '2017-07-12T00:00:00.000000000',
       '2020-07-22T00:00:00.000000000', '2019-11-14T00:00:00.000000000',
       '2013-01-18T00:00:00.000000000', '2019-12-10T00:00:00.000000000',
       '2018-11-28T00:00:00.000000000', '2018-06-27T00:00:00.000000000',
       '2018-11-29T00:00:00.000000000', '2019-11-06T00:00:00.000000000',
       '2019-11-05T00:00:00.000000000', '2018-11-30T00:00:00.000000000',
       '2019-11-01T00:00:00.000000000', '2019-12-12T00:00:00.000000000',
       '2019-12-11T00:00:00.000000000', '2019-11-08T00:00:00.000000000',
       '2019-11-07T00:00:00.000000000', '2019-09-04T00:00:00.000000000',
       '2020-02-11T00:00:00.000000000', '2020-02-18T00:00:00.000000000',
       '2017-12-15T00:00:00.000000000', '2020-11-29T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [37]:
X_train['counter_installation_date'] = X_train['counter_installation_date'] > pd.Timestamp(2020, 9, 1)
X_train.counter_installation_date = X_train.counter_installation_date.replace([True, False], [1, 0])

In [38]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450627 entries, 1963 to 434002
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 450627 non-null  category      
 1   counter_name               450627 non-null  category      
 2   site_id                    450627 non-null  int64         
 3   site_name                  450627 non-null  category      
 4   date                       450627 non-null  datetime64[ns]
 5   counter_installation_date  450627 non-null  int64         
 6   counter_technical_id       450627 non-null  category      
 7   latitude                   450627 non-null  float64       
 8   longitude                  450627 non-null  float64       
 9   temp                       450627 non-null  float64       
 10  dwpt                       450627 non-null  float64       
 11  rhum                       450627 non-null  int64

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder

date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[["date"]]).columns.tolist()

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name", "season"]

preprocessor = ColumnTransformer(
    [
        ("date", OneHotEncoder(handle_unknown="ignore"), date_cols),
        ("cat", categorical_encoder, categorical_cols),
    ]
)

regressor = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs = -1, max_depth=25, bootstrap=False)

pipe = make_pipeline(date_encoder, preprocessor, regressor)
pipe.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

print(
    f"Train set, RMSE={mean_squared_error(y_train, pipe.predict(X_train), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_test, pipe.predict(X_test), squared=False):.2f}"
)

In [42]:
pipe.steps[2][1].feature_importances_.shape

(166,)